In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
from sklearn.metrics import mean_squared_error
from sklearn.ensemble  import RandomForestRegressor
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
train = pd.read_csv("dataset/train.csv")
test = pd.read_csv("dataset/test.csv")
#print(train.describe(include="all"))
pd.options.display.max_columns = None

string_columns = train.select_dtypes(include='object')
unique_counts = string_columns.apply(lambda col: col.unique())

# Mapping: new data frames
mapping = {}
for col in string_columns:
    unique_values = string_columns[col].dropna().unique()
    mapping[col] = {value: i for i, value in enumerate(unique_values)}
    
for col, map_dict in mapping.items():
    train[col] = train[col].map(map_dict)

# Fill all NaN = mean
for col in train.columns: 
    train[col].fillna(round(train[col].mean()), inplace=True)
    
# print(train.describe(include="all"))
        
# train['LotFrontage'].fillna(train['LotFrontage'].mean(), inplace=True)
# To check: columns_with_nan = df.columns[df.isnull().any()]
columns_with_nan = train.columns[train.isnull().any()]
print(columns_with_nan)

X = train.drop('SalePrice', axis=1)

y = train["SalePrice"]

Index([], dtype='object')


In [5]:
model = RandomForestRegressor()

string_columns = test.select_dtypes(include='object')
unique_counts = string_columns.apply(lambda col: col.unique())

mapping = {}
for col in string_columns:
    unique_values = string_columns[col].dropna().unique()
    mapping[col] = {value: i for i, value in enumerate(unique_values)}
    
for col, map_dict in mapping.items():
    test[col] = test[col].map(map_dict)

# Fill all NaN = mean
for col in test.columns: 
    test[col].fillna(round(test[col].mean()), inplace=True)  

# Fit model
model.fit(X, y)

# Make predictions on the test set
y_pred = model.predict(test)

# Calculate the Mean Squared Error
# mse = mean_squared_error(y, y_pred)
# rmse = np.sqrt(mse)

string_columns = test.select_dtypes(include='object')
unique_counts = string_columns.apply(lambda col: col.nunique())
for col, count in unique_counts.items():
    print(f"{col}: {count}")


In [9]:
submission = pd.DataFrame({
    'Id': test['Id'],
    'SalePrice': y_pred
})

# Save the submission DataFrame to a CSV file
submission.to_csv('submission.csv', index=False)